In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error, log_loss
pd.options.display.precision = 15
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
import time
import datetime
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import json
import copy
import pickle

#-------------------------------get data and label-------------------#
train = pd.read_csv('/data/train_base.csv')
test = pd.read_csv('/data/testa_base.csv')
train_fea = pd.read_csv('train_trx_fea.csv')
test_fea = pd.read_csv('test_trx_fea.csv')
# train_trx_emb = pd.read_csv('train_trx_operation_emebdding.csv')
# test_trx_emb = pd.read_csv('test_trx_trx_operation_emebdding.csv')
train_fea_view = pd.read_csv('train_view_fea.csv')
test_fea_view = pd.read_csv('test_view_fea.csv')
view_emb = pd.read_csv('all_view_emb.csv')
#w2v_view_emb = pd.read_csv('all_view_w2v_emb.csv')
#trx_emb = pd.read_csv('all_trx_tfidf_emb.csv')


train = pd.merge(train, train_fea, on=['cust_wid'],how = 'left')
test = pd.merge(test, test_fea, on=['cust_wid'],how = 'left')

train = pd.merge(train, train_fea_view, on=['cust_wid'],how = 'left')
test = pd.merge(test, test_fea_view, on=['cust_wid'],how = 'left')

# train = pd.merge(train, train_trx_emb, on=['cust_wid'],how = 'left')
# test = pd.merge(test, test_trx_emb, on=['cust_wid'],how = 'left')

train = pd.merge(train, view_emb, on=['cust_wid'],how = 'left')
test = pd.merge(test, view_emb, on=['cust_wid'],how = 'left')

# train = pd.merge(train, w2v_view_emb, on=['cust_wid'],how = 'left')
# test = pd.merge(test, w2v_view_emb, on=['cust_wid'],how = 'left')

train = train.dropna(subset = ['age','gdr_cd'])
print(train.shape, test.shape)
train['label'][train.label>=1]=1

In [ ]:
#---------------------------------some functional---------------------------------#
# F2-score
def f2_score(y_true, y_pred):
    return 'f2', fbeta_score(np.round(y_true), np.array(y_pred.label), beta=2), True


def pkl_save(filename,file):
    output = open(filename, 'wb')
    pickle.dump(file, output)
    output.close()

def pkl_load(filename):
    pkl_file = open(filename, 'rb')
    file = pickle.load(pkl_file) 
    pkl_file.close()
    return file

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc


def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True


def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()
    
def train_model_classification(X, X_test, y, params, folds, model_type='lgb', eval_metric='auc', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000, splits=None, n_folds=3, averaging='usual', n_jobs=-1):
    """
    A function to train a variety of classification models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns is None else columns
    n_splits = folds.n_splits if splits is None else n_folds
    X_test = X_test[columns]
    
    # to set up scoring parameters
    metrics_dict = {'auc': {'lgb_metric_name': eval_auc,
                        'catboost_metric_name': 'AUC',
                        'sklearn_scoring_function': metrics.roc_auc_score},
                    }
    
    result_dict = {}
    if averaging == 'usual':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))
        
    elif averaging == 'rank':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))

    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators=n_estimators, n_jobs = n_jobs)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            pkl_save("models/model-{}.lgb".format(fold_n),model)
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=n_estimators, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict_proba(X_test)
        
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=n_estimators, eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function='Logloss')
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        if averaging == 'usual':
            
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
            
            prediction += y_pred.reshape(-1, 1)

        elif averaging == 'rank':
                                  
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
                                  
            prediction += pd.Series(y_pred).rank().values.reshape(-1, 1)        
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_splits
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12))
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
            plt.title('LGB Features (avg over folds)')
            
            result_dict['feature_importance'] = feature_importance
            result_dict['top_columns'] = cols
        
    return result_dict

#---------------------------------------------cat_cols process--------------------------------#
cat_cols = ['gdr_cd','cty_cd'] 
continue_cols = []
for col in train.columns:
    if col not in ['cust_wid','label']+cat_cols:
        continue_cols.append(col)

for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)



#------------------------------------------prepare training data -----------------------------#
n_fold = 5
#folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=5)

#continue_cols+ ['CUST_UID','LABEL']+cat_cols+question_cols+large_cale_cols
X = train.sort_values('cust_wid').drop(['label', 'cust_wid'], axis=1)
y = train.sort_values('cust_wid')['label']
X_test = test.drop(['cust_wid'], axis=1)

def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X = clean_inf_nan(X)
X_test = clean_inf_nan(X_test )


print(len(X.columns))
print(X.shape)

averag = 'usual'#'rank'#
model_name = 'lgb'
params = {'num_leaves': 63,#256,#
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': -1,#7,#15
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.75,#0.9
          "bagging_seed": 11,
          "metric": 'binary_logloss',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          #'categorical_feature': cat_cols
         }

result_dict_lgb = train_model_classification(X=X, X_test=X_test, y=y, params=params, folds=folds, model_type='lgb', eval_metric='auc', plot_feature_importance=True,
                                                      verbose=50, early_stopping_rounds=20, n_estimators=5000, averaging=averag, n_jobs=-1)

In [ ]:
import pickle
def pkl_save(filename,file):
    output = open(filename, 'wb')
    pickle.dump(file, output)
    output.close()

def pkl_load(filename):
    pkl_file = open(filename, 'rb')
    file = pickle.load(pkl_file) 
    pkl_file.close()
    return file


def run_test(X_test,X):
    result_dict = {}
    columns_test = X.columns 
    print(X.columns)
    n_splits = 5
    X_test = X_test[columns_test]

    models = [pkl_load("models/model-{}.lgb".format(idx)) for idx in range(n_splits)]
    prediction = np.zeros((len(X_test), 1))
    for idx,model in enumerate(models):
        y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
        prediction += y_pred.reshape(-1, 1)
    
    prediction /= n_splits
    result_dict['prediction'] = prediction
    return result_dict

result_dict_lgb = run_test(X,X)
result_dict_lgb

In [ ]:
eval_data = train.sort_values('cust_wid')[['cust_wid','label']]
eval_data['pre'] = result_dict_lgb['prediction']
eval_data.head()

In [ ]:
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score
def f2_metric(y_true, y_pred):
    return 'f2', fbeta_score(y_true, y_pred,average='binary',  beta=2), True

max_f2 = 0
max_id = 0
pred = result_dict_lgb['prediction']
for i in np.arange(0,1,0.1):
    
    pred_binary = [1 if x >= i else 0 for x in pred]

    # 计算F2-score
    f2_score_val = f2_metric(np.array(eval_data['label']), pred_binary)
    #print(f2_score_val)
    if max_f2<= f2_score_val[1]:
        max_f2 = f2_score_val[1]
        max_id = i

print("f2_score:",max_f2,'max_id:',max_id)

print(pred.shape,pred)
print(roc_auc_score(np.array(eval_data['label']),pred))